# Experiment 11: Fragmentation-Forcing Interaction

**Phase 4 - Combining Key Findings from Experiments 8 and 10c**

## Background

Two validated findings from Phase 4:

1. **Experiment 8**: Network fragmentation creates asymmetric dynamics
   - At 10% edge retention, tip/recovery ratio = 1.148 (14.8% asymmetry)
   - Fragmentation reduces thermodynamic activity by 96.5%

2. **Experiment 10c**: Active forcing enables recovery
   - Passive recovery: 38.6%
   - Linear relationship: recovery ≈ 0.74 × |f| + 0.51
   - 50% recovery requires |f| ≥ 0.10

## Key Question

**Does fragmentation increase the forcing required for recovery?**

If fragmented networks are harder to recover (higher asymmetry), we expect:
- Steeper forcing-recovery curves at lower retention
- Higher forcing thresholds for 50% recovery
- Possible non-linear interactions

## Experimental Design

| Parameter | Values |
|-----------|--------|
| Edge retention | [1.0, 0.5, 0.25, 0.10] |
| Forcing strength | [0.0, -0.1, -0.2, -0.3, -0.5] |
| Ensemble runs | 20 per condition |
| **Total simulations** | 4 × 5 × 20 = **400** |

## Hypothesis

At 10% retention, the forcing required for 50% recovery will be **2-3× higher** than the intact network.

## 1. Setup and Imports

In [ ]:
import sys
sys.path.insert(0, '/opt/research-local/src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
from pathlib import Path
from netCDF4 import Dataset
from dask.distributed import as_completed

# Core energy-constrained module
from energy_constrained import (
    EnergyConstrainedNetwork,
    EnergyConstrainedCusp,
    GradientDrivenCoupling,
    run_two_phase_experiment,
    get_dask_client,
    fragment_network,
    compute_network_metrics
)

print("Imports successful!")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Connect to Dask cluster
client = get_dask_client()
print(f"Connected to: {client.scheduler_info()['address']}")
print(f"Workers: {len(client.scheduler_info()['workers'])}")
print(f"Total threads: {sum(w['nthreads'] for w in client.scheduler_info()['workers'].values())}")

## 2. Load Amazon Data

In [ ]:
DATA_PATH = Path('/opt/research-local/data/amazon/amazon_adaptation_model/average_network/era5_new_network_data')

def load_amazon_data(year=2003, months=[7, 8, 9]):
    """Load and average Amazon moisture recycling data."""
    all_rain = []
    all_evap = []
    all_network = []
    
    for month in months:
        file_path = DATA_PATH / f'1deg_{year}_{month:02d}.nc'
        if file_path.exists():
            with Dataset(file_path, 'r') as ds:
                all_rain.append(ds.variables['rain'][:])
                all_evap.append(ds.variables['evap'][:])
                all_network.append(ds.variables['network'][:])
    
    return {
        'rain': np.mean(all_rain, axis=0),
        'evap': np.mean(all_evap, axis=0),
        'network': np.mean(all_network, axis=0),
        'n_cells': len(all_rain[0])
    }

amazon_data = load_amazon_data(year=2003)
print(f"Loaded Amazon data: {amazon_data['n_cells']} cells")

## 3. Experiment Configuration

In [ ]:
# Experiment 11: Fragmentation × Forcing Interaction

SWEEP_CONFIG = {
    # Two-dimensional sweep
    'retention_values': [1.0, 0.5, 0.25, 0.10],
    'forcing_values': np.array([0.0, -0.10, -0.20, -0.30, -0.50]),
    'n_runs_per_condition': 20,
    
    # Network parameters
    'n_cells': 50,
    'min_flow': 1.0,
    'barrier_height': 0.2,
    
    # Two-phase simulation parameters
    'cascade_duration': 200,
    'recovery_duration': 800,
    'dt': 0.5,
    'cascade_sigma': 0.06,
    'cascade_alpha': 1.5,
    'recovery_sigma': 0.04,
    'recovery_alpha': 2.0,  # Gaussian
    
    # Fragmentation method
    'fragmentation_method': 'random',
    
    # Seeds
    'base_seed': 42,
}

n_retention = len(SWEEP_CONFIG['retention_values'])
n_forcing = len(SWEEP_CONFIG['forcing_values'])
n_runs = SWEEP_CONFIG['n_runs_per_condition']
total_sims = n_retention * n_forcing * n_runs

print("=" * 60)
print("EXPERIMENT 11: FRAGMENTATION × FORCING INTERACTION")
print("=" * 60)
print(f"Retention levels: {SWEEP_CONFIG['retention_values']}")
print(f"Forcing values: {list(SWEEP_CONFIG['forcing_values'])}")
print(f"Runs per condition: {n_runs}")
print(f"Total simulations: {total_sims}")
print(f"\nEstimated runtime: ~{total_sims * 2 / 60:.0f} minutes on 14 workers")

## 4. Network Creation Functions

In [ ]:
def create_amazon_network(data, config, seed=42):
    """Create 50-cell Amazon network."""
    np.random.seed(seed)
    
    network_matrix = data['network']
    n_cells = config['n_cells']
    min_flow = config['min_flow']
    barrier_height = config['barrier_height']
    
    total_flow = network_matrix.sum(axis=0) + network_matrix.sum(axis=1)
    top_indices = np.argsort(total_flow)[-n_cells:]
    
    net = EnergyConstrainedNetwork()
    
    for i, idx in enumerate(top_indices):
        element = EnergyConstrainedCusp(
            a=-1.0, b=1.0, c=0.0, x_0=0.0,
            barrier_height=barrier_height,
            dissipation_rate=0.1
        )
        net.add_element(f'cell_{i}', element)
    
    n_edges = 0
    for i, idx_i in enumerate(top_indices):
        for j, idx_j in enumerate(top_indices):
            if i != j:
                flow = network_matrix[idx_i, idx_j]
                if flow > min_flow:
                    coupling = GradientDrivenCoupling(
                        conductivity=flow / 100.0,
                        state_coupling=0.1
                    )
                    net.add_coupling(f'cell_{i}', f'cell_{j}', coupling)
                    n_edges += 1
    
    return net, top_indices

# Create baseline network
baseline_network, selected_cells = create_amazon_network(amazon_data, SWEEP_CONFIG)
print(f"Baseline network: {baseline_network.n_elements} nodes, {baseline_network.number_of_edges()} edges")

# Test fragmentation
print("\nFragmentation test:")
for retention in SWEEP_CONFIG['retention_values']:
    if retention == 1.0:
        frag_net = baseline_network
    else:
        frag_net = fragment_network(
            baseline_network, retention,
            method=SWEEP_CONFIG['fragmentation_method'],
            seed=42
        )
    metrics = compute_network_metrics(frag_net)
    print(f"  {retention*100:3.0f}% retention: {metrics['n_edges']:4d} edges, density={metrics['density']:.4f}")

## 5. Worker Function

In [ ]:
def run_fragmentation_forcing_experiment(args):
    """
    Worker function for a single fragmentation + forcing experiment.
    
    Parameters are passed as a tuple for compatibility with client.map().
    """
    network_bytes, retention, forcing, config, seed = args
    
    import sys
    import numpy as np
    import pickle
    
    if '/opt/research-local/src' not in sys.path:
        sys.path.insert(0, '/opt/research-local/src')
    
    from energy_constrained.solvers import run_two_phase_experiment
    from energy_constrained import fragment_network
    
    # Reconstruct network
    baseline_network = pickle.loads(network_bytes)
    np.random.seed(seed)
    
    # Apply fragmentation
    if retention < 1.0:
        network = fragment_network(
            baseline_network, retention,
            method=config['fragmentation_method'],
            seed=seed  # Different fragmentation pattern per run
        )
    else:
        network = baseline_network
    
    # Run two-phase experiment
    result = run_two_phase_experiment(
        network=network,
        cascade_duration=config['cascade_duration'],
        recovery_duration=config['recovery_duration'],
        dt=config['dt'],
        cascade_sigma=config['cascade_sigma'],
        cascade_alpha=config['cascade_alpha'],
        recovery_sigma=config['recovery_sigma'],
        recovery_alpha=config['recovery_alpha'],
        recovery_forcing=forcing,
        seed=seed
    )
    
    # Compute additional metrics
    pct_time_tipped = np.mean(result.x_full > 0) * 100
    
    n_cells = result.x_full.shape[1]
    n_tip_events = 0
    n_recover_events = 0
    
    for j in range(n_cells):
        x_traj = result.x_full[:, j]
        signs = np.sign(x_traj)
        sign_changes = np.diff(signs)
        n_tip_events += np.sum(sign_changes > 0)
        n_recover_events += np.sum(sign_changes < 0)
    
    return {
        'retention': retention,
        'forcing': forcing,
        'seed': seed,
        'recovery_fraction': result.metrics['recovery_fraction'],
        'pct_tipped_cascade': result.metrics['pct_tipped_at_cascade_end'],
        'n_permanent_tips': result.metrics['n_permanent_tips'],
        'final_pct_tipped': result.metrics['final_pct_tipped'],
        'pct_time_tipped': pct_time_tipped,
        'n_tip_events': n_tip_events,
        'n_recover_events': n_recover_events,
        'mean_recovery_time': result.metrics['mean_recovery_time'],
    }

print("Worker function defined.")

## 6. Run Two-Dimensional Sweep

In [ ]:
# Serialize and scatter baseline network
network_bytes = pickle.dumps(baseline_network)
print(f"Network serialized: {len(network_bytes) / 1024:.1f} KB")

network_future = client.scatter(network_bytes, broadcast=True)
print("Network broadcast to all workers")

# Build task arguments
print("\n" + "=" * 60)
print("EXPERIMENT 11: Starting Fragmentation × Forcing Sweep")
print("=" * 60)
start_time = time.time()

task_args = []
task_idx = 0
for r_idx, retention in enumerate(SWEEP_CONFIG['retention_values']):
    for f_idx, forcing in enumerate(SWEEP_CONFIG['forcing_values']):
        for run_idx in range(SWEEP_CONFIG['n_runs_per_condition']):
            seed = SWEEP_CONFIG['base_seed'] + r_idx * 10000 + f_idx * 1000 + run_idx
            task_args.append((network_bytes, float(retention), float(forcing), SWEEP_CONFIG, seed))
            task_idx += 1

print(f"Generated {len(task_args)} task arguments")

# Submit all tasks at once using client.map()
futures = client.map(run_fragmentation_forcing_experiment, task_args)
print(f"Submitted {len(futures)} tasks")

# Collect results with progress tracking
all_results = []
completed_by_condition = {}

print("\nProgress:")
for i, future in enumerate(as_completed(futures)):
    result = future.result()
    all_results.append(result)
    
    # Track progress by condition
    key = (result['retention'], result['forcing'])
    completed_by_condition[key] = completed_by_condition.get(key, 0) + 1
    
    if (i + 1) % 40 == 0:
        elapsed = time.time() - start_time
        rate = (i + 1) / elapsed
        remaining = (len(futures) - i - 1) / rate
        print(f"  Completed {i+1}/{len(futures)} ({100*(i+1)/len(futures):.1f}%) "
              f"- {elapsed:.0f}s elapsed, ~{remaining:.0f}s remaining")

elapsed = time.time() - start_time
print(f"\n" + "=" * 60)
print(f"COMPLETE: {len(all_results)} simulations in {elapsed:.1f}s ({elapsed/60:.1f} min)")
print("=" * 60)

## 7. Results Aggregation

In [ ]:
df = pd.DataFrame(all_results)
print(f"Results shape: {df.shape}")

df = df.sort_values(['retention', 'forcing', 'seed']).reset_index(drop=True)
df.head(10)

In [ ]:
# Summary statistics by retention and forcing
summary = df.groupby(['retention', 'forcing']).agg({
    'recovery_fraction': ['mean', 'std', 'min', 'max'],
    'n_permanent_tips': ['mean', 'std'],
    'pct_tipped_cascade': ['mean', 'std'],
    'n_tip_events': ['mean'],
    'n_recover_events': ['mean'],
}).round(4)

summary.columns = ['_'.join(col) for col in summary.columns]
summary = summary.reset_index()

# Add tip/recovery ratio
summary['tip_recovery_ratio'] = summary['n_tip_events_mean'] / summary['n_recover_events_mean']

print("=" * 90)
print("FRAGMENTATION × FORCING SUMMARY")
print("=" * 90)
print(summary[['retention', 'forcing', 'recovery_fraction_mean', 'recovery_fraction_std',
               'n_permanent_tips_mean', 'tip_recovery_ratio']].to_string(index=False))

## 8. Primary Visualization: Interaction Effects

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Color palette for retention levels
colors = plt.cm.viridis(np.linspace(0, 0.8, len(SWEEP_CONFIG['retention_values'])))
retention_labels = [f"{r*100:.0f}%" for r in SWEEP_CONFIG['retention_values']]

# Panel 1: Recovery Fraction vs Forcing (by retention)
ax = axes[0, 0]
for i, retention in enumerate(SWEEP_CONFIG['retention_values']):
    subset = summary[summary['retention'] == retention]
    ax.errorbar(
        -subset['forcing'].values,
        subset['recovery_fraction_mean'].values,
        yerr=subset['recovery_fraction_std'].values,
        marker='o', capsize=4, linewidth=2, markersize=8,
        color=colors[i], label=retention_labels[i]
    )
ax.axhline(0.5, color='gray', linestyle='--', alpha=0.7, label='50% recovery')
ax.set_xlabel('Restoration Forcing Strength (|f|)', fontsize=12)
ax.set_ylabel('Recovery Fraction', fontsize=12)
ax.set_title('Recovery vs Forcing (by Fragmentation)', fontsize=14)
ax.set_ylim(0, 1.05)
ax.legend(title='Edge Retention')
ax.grid(True, alpha=0.3)

# Panel 2: Tip/Recovery Ratio vs Forcing (by retention)
ax = axes[0, 1]
for i, retention in enumerate(SWEEP_CONFIG['retention_values']):
    subset = summary[summary['retention'] == retention]
    ax.plot(
        -subset['forcing'].values,
        subset['tip_recovery_ratio'].values,
        marker='s', linewidth=2, markersize=8,
        color=colors[i], label=retention_labels[i]
    )
ax.axhline(1.0, color='gray', linestyle='--', alpha=0.7, label='Symmetric')
ax.set_xlabel('Restoration Forcing Strength (|f|)', fontsize=12)
ax.set_ylabel('Tip/Recovery Event Ratio', fontsize=12)
ax.set_title('Asymmetry vs Forcing (by Fragmentation)', fontsize=14)
ax.legend(title='Edge Retention')
ax.grid(True, alpha=0.3)

# Panel 3: Heatmap of Recovery Fraction
ax = axes[1, 0]
pivot = summary.pivot(index='retention', columns='forcing', values='recovery_fraction_mean')
im = ax.imshow(pivot.values, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
ax.set_xticks(range(len(pivot.columns)))
ax.set_xticklabels([f"{-f:.1f}" for f in pivot.columns])
ax.set_yticks(range(len(pivot.index)))
ax.set_yticklabels([f"{r*100:.0f}%" for r in pivot.index])
ax.set_xlabel('Forcing Strength (|f|)', fontsize=12)
ax.set_ylabel('Edge Retention', fontsize=12)
ax.set_title('Recovery Fraction Heatmap', fontsize=14)

# Add values to heatmap
for i in range(len(pivot.index)):
    for j in range(len(pivot.columns)):
        val = pivot.values[i, j]
        color = 'white' if val < 0.5 else 'black'
        ax.text(j, i, f"{val:.2f}", ha='center', va='center', color=color, fontsize=10)

plt.colorbar(im, ax=ax, label='Recovery Fraction')

# Panel 4: Permanent Tips Heatmap
ax = axes[1, 1]
pivot_tips = summary.pivot(index='retention', columns='forcing', values='n_permanent_tips_mean')
im2 = ax.imshow(pivot_tips.values, cmap='Reds', aspect='auto')
ax.set_xticks(range(len(pivot_tips.columns)))
ax.set_xticklabels([f"{-f:.1f}" for f in pivot_tips.columns])
ax.set_yticks(range(len(pivot_tips.index)))
ax.set_yticklabels([f"{r*100:.0f}%" for r in pivot_tips.index])
ax.set_xlabel('Forcing Strength (|f|)', fontsize=12)
ax.set_ylabel('Edge Retention', fontsize=12)
ax.set_title('Permanent Tips Heatmap', fontsize=14)

# Add values
for i in range(len(pivot_tips.index)):
    for j in range(len(pivot_tips.columns)):
        val = pivot_tips.values[i, j]
        color = 'white' if val > 20 else 'black'
        ax.text(j, i, f"{val:.1f}", ha='center', va='center', color=color, fontsize=10)

plt.colorbar(im2, ax=ax, label='Permanent Tips')

plt.tight_layout()
plt.savefig('/workspace/data/exp11_fragmentation_forcing.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nPlot saved to /workspace/data/exp11_fragmentation_forcing.png")

## 9. Critical Forcing by Retention Level

In [ ]:
def find_critical_forcing(summary_df, retention, recovery_threshold=0.5):
    """
    Find minimum forcing needed to achieve recovery_threshold at given retention.
    """
    subset = summary_df[summary_df['retention'] == retention].sort_values('forcing')
    forcings = subset['forcing'].values
    recoveries = subset['recovery_fraction_mean'].values
    
    # Find crossing point
    for i in range(len(forcings) - 1):
        if recoveries[i] >= recovery_threshold > recoveries[i+1]:
            # Linear interpolation
            slope = (recoveries[i+1] - recoveries[i]) / (forcings[i+1] - forcings[i])
            if slope != 0:
                critical = forcings[i] + (recovery_threshold - recoveries[i]) / slope
                return critical
    
    # Check boundary cases
    if recoveries.max() < recovery_threshold:
        return None  # Never reaches threshold
    elif recoveries.min() >= recovery_threshold:
        return forcings.max()  # Even weakest forcing is enough
    
    return None


print("=" * 70)
print("CRITICAL FORCING BY RETENTION LEVEL")
print("=" * 70)

critical_forcings = {}
for retention in SWEEP_CONFIG['retention_values']:
    critical = find_critical_forcing(summary, retention, 0.50)
    critical_forcings[retention] = critical
    
    if critical is not None:
        print(f"\n{retention*100:.0f}% retention: |f| = {-critical:.3f} for 50% recovery")
    else:
        # Check if always above or below
        subset = summary[summary['retention'] == retention]
        max_rec = subset['recovery_fraction_mean'].max()
        if max_rec >= 0.5:
            print(f"\n{retention*100:.0f}% retention: 50% recovery achieved even at f=0")
        else:
            print(f"\n{retention*100:.0f}% retention: 50% recovery NOT achieved (max={max_rec:.2f})")

In [ ]:
# Calculate forcing multiplier relative to intact network
print("\n" + "=" * 70)
print("FORCING REQUIREMENT MULTIPLIER")
print("=" * 70)

baseline_forcing = critical_forcings.get(1.0)
if baseline_forcing is not None:
    print(f"\nBaseline (100% retention): |f| = {-baseline_forcing:.3f}")
    print("\nMultiplier relative to baseline:")
    for retention in SWEEP_CONFIG['retention_values']:
        critical = critical_forcings.get(retention)
        if critical is not None and baseline_forcing is not None and baseline_forcing != 0:
            multiplier = critical / baseline_forcing
            print(f"  {retention*100:.0f}% retention: {multiplier:.2f}× baseline forcing")
        elif critical is None:
            print(f"  {retention*100:.0f}% retention: N/A (threshold not reached)")
else:
    print("Baseline (100%) achieves 50% recovery with no forcing")
    for retention in SWEEP_CONFIG['retention_values']:
        if retention < 1.0:
            critical = critical_forcings.get(retention)
            if critical is not None:
                print(f"  {retention*100:.0f}% retention requires |f| = {-critical:.3f}")

## 10. Interaction Analysis

In [ ]:
# Fit linear models for each retention level
print("=" * 70)
print("LINEAR FORCING-RECOVERY RELATIONSHIP BY RETENTION")
print("=" * 70)

fits = {}
for retention in SWEEP_CONFIG['retention_values']:
    subset = summary[summary['retention'] == retention]
    forcings = -subset['forcing'].values  # Use |f|
    recoveries = subset['recovery_fraction_mean'].values
    
    # Linear fit
    if len(forcings) > 1:
        slope, intercept = np.polyfit(forcings, recoveries, 1)
        fits[retention] = {'slope': slope, 'intercept': intercept}
        print(f"\n{retention*100:.0f}% retention:")
        print(f"  recovery ≈ {slope:.3f} × |f| + {intercept:.3f}")
        print(f"  Each 0.1 increase in |f| adds {slope*0.1:.1%} recovery")

# Compare slopes
print("\n" + "=" * 70)
print("FORCING EFFECTIVENESS COMPARISON")
print("=" * 70)
if fits:
    baseline_slope = fits.get(1.0, {}).get('slope', 0)
    if baseline_slope != 0:
        for retention in SWEEP_CONFIG['retention_values']:
            if retention in fits:
                slope = fits[retention]['slope']
                efficiency = slope / baseline_slope * 100
                print(f"  {retention*100:.0f}% retention: {efficiency:.1f}% as effective as intact network")

## 11. Key Findings Summary

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 11: KEY FINDINGS")
print("=" * 70)

# Best and worst conditions
best_idx = summary['recovery_fraction_mean'].idxmax()
worst_idx = summary['recovery_fraction_mean'].idxmin()

best = summary.loc[best_idx]
worst = summary.loc[worst_idx]

print(f"""
1. BEST RECOVERY CONDITION:
   Retention: {best['retention']*100:.0f}%
   Forcing: f = {best['forcing']:.2f}
   Recovery: {best['recovery_fraction_mean']:.1%} ± {best['recovery_fraction_std']:.1%}

2. WORST RECOVERY CONDITION:
   Retention: {worst['retention']*100:.0f}%
   Forcing: f = {worst['forcing']:.2f}
   Recovery: {worst['recovery_fraction_mean']:.1%} ± {worst['recovery_fraction_std']:.1%}

3. FRAGMENTATION-FORCING INTERACTION:
""")

# Compare passive recovery across retention levels
print("   Passive recovery (f=0) by retention:")
for retention in SWEEP_CONFIG['retention_values']:
    passive = summary[(summary['retention'] == retention) & (summary['forcing'] == 0)]
    if len(passive) > 0:
        rec = passive['recovery_fraction_mean'].values[0]
        print(f"     {retention*100:.0f}% retention: {rec:.1%}")

# Compare maximum forcing recovery
max_forcing = SWEEP_CONFIG['forcing_values'].min()  # Most negative = strongest
print(f"\n   Maximum forcing (f={max_forcing}) by retention:")
for retention in SWEEP_CONFIG['retention_values']:
    max_f = summary[(summary['retention'] == retention) & (summary['forcing'] == max_forcing)]
    if len(max_f) > 0:
        rec = max_f['recovery_fraction_mean'].values[0]
        print(f"     {retention*100:.0f}% retention: {rec:.1%}")

## 12. Save Results

In [ ]:
df.to_csv('/workspace/data/experiment11_fragmentation_forcing.csv', index=False)
print(f"Full results saved to /workspace/data/experiment11_fragmentation_forcing.csv")

summary.to_csv('/workspace/data/experiment11_summary.csv', index=False)
print(f"Summary saved to /workspace/data/experiment11_summary.csv")

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 11 COMPLETE")
print("=" * 70)

print(f"""
CONFIGURATION:
- Retention levels: {SWEEP_CONFIG['retention_values']}
- Forcing values: {list(SWEEP_CONFIG['forcing_values'])}
- Runs per condition: {SWEEP_CONFIG['n_runs_per_condition']}
- Total simulations: {len(df)}
- Runtime: {elapsed:.1f}s ({elapsed/60:.1f} min)

FILES GENERATED:
- /workspace/data/experiment11_fragmentation_forcing.csv
- /workspace/data/experiment11_summary.csv
- /workspace/data/exp11_fragmentation_forcing.png

KEY CONCLUSIONS:
- [Fill after running experiment]
""")